In [1]:
import cv2
import os
import numpy as np
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [27]:
def load_and_preprocess_images(image_dir, img_size=(64, 64)):
    images = []
    labels = []
    for label in ['infected', 'uninfected']:
        path = os.path.join(image_dir, label)
        class_num = 0 if label == 'uninfected' else 1
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img_gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                resized_array = cv2.resize(img_gray, img_size)
                images.append(resized_array)
                labels.append(class_num)
            except Exception as e:
                pass
    images = np.array(images).reshape(-1, img_size[0], img_size[1])
    images = images / 255.0
    return images, np.array(labels)

def extract_hog_features(images):
    hog_features = []
    for image in images:
        feature = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
        hog_features.append(feature)
    return np.array(hog_features)

image_dir = 'C:/Users/PMLS/Downloads/Compressed/cell_images'
X, y = load_and_preprocess_images(image_dir)
X_features = extract_hog_features(X)


Training

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.84


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [24]:
def detect_malaria(image_path, model, img_size=(64, 64)):
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    oimg = cv2.imread(image_path)
    
    # Check if the image was successfully loaded
    if img is None:
        print(f"Error: Unable to read image at path {image_path}")
        return "Error: Unable to read image"
    
    # Display the original image
    '''cv2.imshow("Original Image", img)
    cv2.waitKey(0)  # Wait for a key press to close the image window
    cv2.destroyAllWindows()  # Close the image window**/
    '''
    # Resize the image
    try:
        img_resized = cv2.resize(img, img_size)
    except cv2.error as e:
        print(f"Error resizing image: {e}")
        return "Error resizing image"
    
    # Display the resized image
    cv2.imshow("Resized Image", oimg)
    cv2.waitKey(0)  # Wait for a key press to close the image window
    cv2.destroyAllWindows()  # Close the image window

    # Normalize and reshape the image
    img_resized = img_resized / 255.0
    img_resized = img_resized.reshape(1, img_size[0], img_size[1])

    # Extract HOG features
    features = hog(img_resized[0], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    features = features.reshape(1, -1)
    
    # Predict using the trained model
    prediction = model.predict(features)
    
    # Return the prediction result
    return 'infected' if prediction == 1 else 'uninfected'

# Example usage
image_path = 'C:/Users/PMLS/Downloads/Compressed/cell_images/002.png'
result = detect_malaria(image_path, model)
print(f'The image is {result}.')

The image is infected.


Saving the model

SVM MODEL

In [8]:
import joblib

In [13]:
model = joblib.load("SVM_model.joblib")

In [11]:
X_train, X_test, y_train, y_test = joblib.load("data_splits.joblib")

In [14]:
y_pred = model.predict(X_test)

In [16]:
accuracy = accuracy_score(y_test,y_pred)
print(f'Loaded model accuracy: {accuracy:.2f}')


Loaded model accuracy: 0.84
